In [1]:
#載入資料
import pandas as pd  
import numpy as np  
df = pd.read_csv('iris.csv')

In [2]:
from sklearn.model_selection import train_test_split
#from sklearn.neighbors import KNeighborsClassifier

In [3]:
#split data and train
X = df.iloc[:,:4]
y = df['species'].replace({'setosa':0,'versicolor':1,'virginica':2}).copy().values

In [4]:
#Accuracy
def Accuracy(y_pred,y_test):
    count = 0
    for i in range(len(y_pred)):
        if y_pred[i] == y_test[i]:
            count = count + 1
    return count / len(y_pred)

In [5]:
#fa
from numpy.linalg import eig,inv
def to_fa_3(X_train, X_test):
    k0 = 1
    x_mean = X_train.mean().values
    C = X_train - x_mean
    C_covariance = C.cov().values
    eigenvalues, eigenvectors = eig(C_covariance)
    w_vector = []
    for i in range(len(eigenvalues)):
        w_vector.append(np.sqrt(eigenvalues)[i] * eigenvectors[i])
    w_vector = np.asarray(w_vector)
    Vt_V = w_vector.T.dot(w_vector)
    V_final = inv(Vt_V).dot(w_vector.T)
    
    C_test = X_test - x_mean
    return V_final.dot(C.T).T[:,:3],V_final.dot(C_test.T).T[:,:3]

In [6]:
#5-nn手寫算法
def knn_5(X_train,X_test,y_train):
    pre = []
    for i in X_test:
        dist = []
        for j in range(len(X_train)):
            dist.append((np.linalg.norm(i-X_train[j]),y_train[j]))
        sort_dist = sorted(dist, key=lambda x:x[0])
        vote = []
        for i in sort_dist[:5]:
            vote.append(i[1])
        pre.append(max(vote,key=vote.count))
    return pre

In [10]:
#train
Accuracy_list_Fa = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)#random split
    #FA
    Fa_X_train,Fa_X_test = to_fa_3(X_train, X_test)
# 下3行套件測試用，因手寫算法執行時間有些久
#    KNNclassifier_Fa = KNeighborsClassifier(n_neighbors=5,p = 2)#p=2 Euclidean
#    KNNclassifier_Fa.fit(Fa_X_train, y_train)
#    Fa_y_pred = KNNclassifier_Fa.predict(Fa_X_test)
    #手寫算法
    Fa_y_pred = knn_5(Fa_X_train,Fa_X_test,y_train)
    Accuracy_list_Fa.append(Accuracy(Fa_y_pred,y_test))

print ('Each accuracy with FA:', Accuracy_list_Fa)
print ('Average accuracy with FA:', sum(Accuracy_list_Fa) / 10)

Each accuracy with FA: [0.9555555555555556, 0.9555555555555556, 0.9777777777777777, 0.9555555555555556, 0.9555555555555556, 0.9777777777777777, 0.9333333333333333, 0.9555555555555556, 0.9333333333333333, 0.9777777777777777]
Average accuracy with FA: 0.9577777777777777
